###Retinotopy

In [1]:
from PIL import Image
from PIL import ImageDraw as Draw
from PIL import ImageFilter
from skimage.filter import gaussian_filter
import numpy as np
from glob import glob
from os import path
from object_parsing.src.image_objects import make_a_blank
import scipy.io as sio

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
savedir = '/Users/tnaselar/Data/Presentation/predictive.coding.match.nonmatch/prf_stim/'
frames_per_pattern = 2 ##>= 1


In [3]:
import numpy as np, h5py 
stim_data = h5py.File('/Users/tnaselar/Data/experiments/prfStimuli/stimuli.mat','r') 

In [4]:
patterns = stim_data.get('patterns')
print patterns.shape

(100, 3, 768, 768)


In [5]:
masks = stim_data.get('masks')
print masks.shape[-1]

768


In [6]:
multibarindices = stim_data.get('multibarindices')
wedgeringindices = stim_data.get('wedgeringindices')

In [7]:
masks.shape[0]

2580

In [8]:
back_lum = 137
background = make_a_blank(masks.shape[-1],back_lum,'RGB')

In [9]:
def format_pattern(patterns,dx):
    return np.transpose(np.rollaxis(patterns[dx,0:,0:,0:].squeeze(),0,start=3),axes = [1,0,2])

def format_masks(masks,dx):
    return np.transpose(np.rollaxis(masks[dx,0:,0:].squeeze(),0,start=2),axes = [1,0])

In [10]:
print format_pattern(patterns,99).shape
print format_masks(masks,99).shape

(768, 768, 3)
(768, 768)


In [11]:
##loop over the mask indices, selecting random pattern each iteration
n_patterns = patterns.shape[0]
n_masks = masks.shape[0]
for mdx in range(n_masks,n_masks+1):
    if not remainder(mdx,50):
        print 'constructing stimuli: %0.6d' %(mdx)
    cur_mask = Image.fromarray(format_masks(masks,mdx-1))
    pat_dx = permutation(range(n_patterns))[0]
    pat = Image.fromarray(format_pattern(patterns, pat_dx),'RGB')
    cur_stim = Image.composite(pat,background,cur_mask)
    cur_mask.save(savedir+'mask_%0.6d.png' %(mdx))
    cur_stim.save(savedir+'stim_%0.6d.png' %(mdx))



In [12]:
background.save(savedir+'stim_%0.6d.png' %(0))

In [12]:
def print_frame_sequence(frame_sequence,print_to):  #+'movie_'+str(mm))
    with open(print_to,'w') as thefile:
        for item in frame_sequence:
          thefile.write("%s\n" % item)                

In [13]:
##bar frame_file
bar_list = []
for mbx in multibarindices:
    bar_list.append('stim_%0.6d.png' %(mbx))
print_frame_sequence(bar_list, savedir+'bar_frame_file.txt')

##wedge frame
wedge_list = []
for mwx in wedgeringindices:
    wedge_list.append('stim_%0.6d.png' %(mwx))
print_frame_sequence(wedge_list,savedir+'wedge_frame_file.txt')